In [1]:
# Neural Network for Bank Data
# Data from http://archive.ics.uci.edu/ml/datasets/Bank+Marketing

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from keras.losses import binary_crossentropy
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
bank = pd.read_csv("data/bank/bank-full.csv", sep=';')

In [3]:
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


Alright we need to encode this because non-numeric data is as good as garbage

In [4]:
def baggize(values):
    sack = {}
    for v in values:
        if v not in sack.keys():
            sack[v] = len(sack)
    kcas = dict((v, k) for (k, v) in sack.items())
    return {'sack': sack, 'kcas': kcas}

In [5]:
columns = bank.columns
columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [6]:
sacks = {}
for c in columns:
    if bank[c].dtype == 'O':
        vals = bank[c].values
        result = baggize(vals)
        sacks[c] = result
        sack = result["sack"]
        bank[c] = [sack[v] for v in vals]

In [7]:
bank

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,0,0,0,0,2143,0,0,0,5,0,261,1,-1,0,0,0
1,44,1,1,1,0,29,0,0,0,5,0,151,1,-1,0,0,0
2,33,2,0,1,0,2,0,1,0,5,0,76,1,-1,0,0,0
3,47,3,0,2,0,1506,0,0,0,5,0,92,1,-1,0,0,0
4,33,4,1,2,0,1,1,0,0,5,0,198,1,-1,0,0,0
5,35,0,0,0,0,231,0,0,0,5,0,139,1,-1,0,0,0
6,28,0,1,0,0,447,0,1,0,5,0,217,1,-1,0,0,0
7,42,2,2,0,1,2,0,0,0,5,0,380,1,-1,0,0,0
8,58,5,0,3,0,121,0,0,0,5,0,50,1,-1,0,0,0
9,43,1,1,1,0,593,0,0,0,5,0,55,1,-1,0,0,0


In [8]:
ys = bank["y"].values.reshape(-1,1)

In [9]:
xs = bank[[c for c in columns if c != "y"]].values

In [10]:
xs, ys

(array([[ 58,   0,   0, ...,  -1,   0,   0],
        [ 44,   1,   1, ...,  -1,   0,   0],
        [ 33,   2,   0, ...,  -1,   0,   0],
        ...,
        [ 72,   5,   0, ..., 184,   3,   3],
        [ 57,   3,   0, ...,  -1,   0,   0],
        [ 37,   2,   0, ..., 188,  11,   2]]), array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [0]]))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=0.2)

In [12]:
train_len = len(X_train)
inp_shape = X_train[0].shape
out_shape = y_train[0].shape
train_len, inp_shape, out_shape

(36168, (16,), (1,))

In [13]:
model = Sequential()
model.add(Dense(units=16, activation='tanh', input_shape=inp_shape))
model.add(Dense(units=8, activation='tanh', input_shape=(16,)))
model.add(Dense(units=4, activation='tanh', input_shape=(8,)))
model.add(Dense(units=2, activation='tanh', input_shape=(4,)))
model.add(Dense(units=1, activation='tanh', input_shape=(2,)))
tb_callback = TensorBoard(log_dir='./logs/bank')

In [14]:
epochs = 50
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs, batch_size=96,
          callbacks=[tb_callback])

Epoch 1/50
36168/36168 [==============================] - 2s 66us/step - loss: 1.0915 - acc: 0.7884
Epoch 2/50
36168/36168 [==============================] - 2s 65us/step - loss: 0.3772 - acc: 0.8839
Epoch 3/50
36168/36168 [==============================] - 2s 54us/step - loss: 0.3432 - acc: 0.8840
Epoch 4/50
36168/36168 [==============================] - 2s 55us/step - loss: 0.3347 - acc: 0.8840
Epoch 5/50
36168/36168 [==============================] - 2s 56us/step - loss: 0.3329 - acc: 0.8840
Epoch 6/50
36168/36168 [==============================] - 2s 53us/step - loss: 0.3286 - acc: 0.8840
Epoch 7/50
36168/36168 [==============================] - 2s 58us/step - loss: 0.3210 - acc: 0.8840
Epoch 8/50
36168/36168 [==============================] - 2s 58us/step - loss: 0.3228 - acc: 0.8840
Epoch 9/50
36168/36168 [==============================] - 2s 55us/step - loss: 0.3199 - acc: 0.8840
Epoch 10/50
36168/36168 [==============================] - 2s 57us/step - loss: 0.3176 - acc: 0.8840

In [15]:
score = model.evaluate(X_test, y_test, batch_size=64)
score

9043/9043 [==============================] - 0s 27us/step


[0.2723594950381257, 0.8789118655774889]

In [16]:
predicted = model.predict(X_test)

In [17]:
predicted = [0 if p < 0.5 else 1 for p in predicted]

In [18]:
errors = [(y1, y2) for (y1, y2) in zip(predicted, y_test) if y1 != y2]

In [19]:
len(errors)

1094